In [ ]:
# %%
# filename: data_download_mt5_to_parquet.ipynb

# --- 1. Imports and Setup ---
import os
import sys
from datetime import datetime, timedelta

import MetaTrader5 as mt5
import pandas as pd

print("--- Initializing Setup ---")
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
    config_file_path = os.path.join(project_root, "config.ini")

    if project_root not in sys.path:
        sys.path.append(project_root)

    from utils.mt5_connector import MT5Connector

    print("Setup successful. Connector is ready.")
except Exception as e:
    print(f"Error during setup: {e}")
    sys.exit()

In [ ]:
# %%
# --- 2. Download Configuration ---
# Modify the parameters in this cell to download different data.

# Connection settings
TERMINAL_SECTION = "terminal_alpha"
ACCOUNT_SECTION = "account_alpha_demo"

# Data settings
SYMBOL = "EURUSD"
TIMEFRAME_NAME = "M15"
TIMEFRAME_CODE = mt5.TIMEFRAME_M15

# Date calculation
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=365)

# Output path construction
start_str = START_DATE.strftime("%Y-%m-%d")
end_str = END_DATE.strftime("%Y-%m-%d")
OUTPUT_DIR = os.path.join(project_root, "data", "parquet")
os.makedirs(OUTPUT_DIR, exist_ok=True)
FILENAME = f"{SYMBOL}_{TIMEFRAME_NAME}_{start_str}_to_{end_str}.parquet"
FILE_PATH = os.path.join(OUTPUT_DIR, FILENAME)

print("--- Configuration Set ---")
print(f"Symbol: {SYMBOL} ({TIMEFRAME_NAME})")
print(f"Date range: {START_DATE.date()} to {END_DATE.date()}")
print(f"Output file will be: {FILE_PATH}")

In [ ]:
# %%
# --- 3. Execution: Connect, Download, and Save ---
# This cell uses the parameters defined in the cell above.

print("\n--- Starting Data Processing ---")
try:
    # The 'with' statement ensures the connection is properly managed and closed
    with MT5Connector(
        config_path=config_file_path, terminal_section=TERMINAL_SECTION
    ) as mt5_conn:
        print(">>> Terminal Initialized Successfully <<<")

        if not mt5_conn.login_to_account(account_section=ACCOUNT_SECTION):
            raise ConnectionError("Failed to log in to the account.")

        print(">>> Account Login Successful! <<<")

        # 1. EXTRACT
        print("\nFetching data from MT5...")
        rates = mt5.copy_rates_range(SYMBOL, TIMEFRAME_CODE, START_DATE, END_DATE)

        if rates is None or len(rates) == 0:
            raise ValueError(f"No data returned for {SYMBOL} on {TIMEFRAME_NAME}.")

        print(f"Fetched {len(rates)} bars.")

        # 2. TRANSFORM
        print("Transforming data...")
        df = pd.DataFrame(rates)
        df["time"] = pd.to_datetime(df["time"], unit="s")
        df.rename(
            columns={
                "time": "Time",
                "open": "Open",
                "high": "High",
                "low": "Low",
                "close": "Close",
                "tick_volume": "Volume",
            },
            inplace=True,
        )
        df = df[["Time", "Open", "High", "Low", "Close", "Volume"]]

        # 3. LOAD
        print(f"Saving data to {FILE_PATH}...")
        df.to_parquet(FILE_PATH, index=False)

        print("\n>>> Data successfully downloaded and saved! <<<")

except Exception as e:
    print(f"\n>>> AN ERROR OCCURRED: {e} <<<")

finally:
    print("\nProcess finished. MT5 connection has been closed.")